#0. Environment Setting

In [ ]:
import pandas as pd
import numpy as np
import chardet
import glob
import os
from scipy import stats

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.patches as patches


#1. Data Preparation

###1.1 Data import and merge

In [ ]:
# dau file list
dau_files = glob.glob("/content/DAU*.csv")
dau_files

['/content/DAU Identity V(Mar 31, 2018 - Jul 13, 2025, 中国).csv',
 '/content/DAU Justice (May 20, 2023 - Jul 13, 2025, 中国).csv',
 '/content/DAU Genshin Impact (Sep 25, 2020 - Jul 12, 2025, 中国).csv',
 '/content/DAU Zenless Zero Zone (Jul 2, 2024 - Jul 12, 2025, 中国).csv',
 '/content/DAU Life After (Oct 31, 2018 - Jul 13, 2025, 中国).csv',
 '/content/DAU Metal Slug Awakening(Apr 18, 2023 - Jul 13, 2025, 中国).csv',
 '/content/DAU Honor of King  (Oct 25, 2015 - Jul 13, 2025, 中国).csv',
 '/content/DAU Dawn of Kingdoms (Mar 29, 2022 - Jul 13, 2025, 中国).csv',
 '/content/DAU  PUBG Mobile (Feb 8, 2018 - Jul 13, 2025, 中国).csv',
 '/content/DAU Honkai Star Rail (Apr 23, 2023 - Jul 12, 2025, 中国).csv']

In [ ]:
#rev file list
rev_files = glob.glob("/content/revenue*.csv")
rev_files

['/content/revenue Genshin Impact (Sep 25, 2020 - Jul 13, 2025, 中国), 详细.csv',
 '/content/revenue HonKai Star Rail (Apr 23, 2023 - Jul 13, 2025, 中国).csv',
 '/content/revenue Zenless Zero Zone (Jul 2, 2024 - Jul 13, 2025, 中国).csv',
 '/content/revenue Justice (May 20, 2023 - Jul 14, 2025, 中国).csv',
 '/content/revenue PUBG Mobile (Feb 8, 2018 - Jul 14, 2025, 中国).csv',
 '/content/revenue Identity V (Mar 31, 2018 - Jul 14, 2025, 中国).csv',
 '/content/revenue Honor of King(Oct 25, 2015 - Jul 14, 2025, 中国).csv',
 '/content/revenue Dawn of Kingdoms (Mar 29, 2022 - Jul 14, 2025, 中国).csv',
 '/content/revenue Life After (Oct 31, 2018 - Jul 14, 2025, 中国), 详细.csv',
 '/content/revenue Metal Slug Awakening (Apr 18, 2023 - Jul 14, 2025, 中国).csv']

In [ ]:
#Check and try encoding

def smart_read_csv(file_path, try_encodings=None, try_seps=None):
    """
    检测 encoding&分隔符，直到成功解析为多列
    """
    if try_encodings is None:
        try_encodings = ['utf-8', 'utf-16', 'latin1', 'cp949']
    if try_seps is None:
        try_seps = [',', '\t', ';']

    # try charde
    with open(file_path, 'rb') as f:
        raw_data = f.read()
        detect_enc = chardet.detect(raw_data)['encoding']
        if detect_enc and detect_enc not in try_encodings:
            try_encodings.insert(0, detect_enc)

    # try all
    for enc in try_encodings:
        for sep in try_seps:
            try:
                df = pd.read_csv(file_path, encoding=enc, sep=sep)
                if df.shape[1] > 1:
                    print(f" 成功读取: {file_path}, encoding={enc}, sep='{sep}', shape={df.shape}")
                    return df
            except Exception as e:
                continue

    raise Exception(f"无法解析文件: {file_path}")


#apply
df_dau = pd.concat([smart_read_csv(f) for f in dau_files], ignore_index=True)
df_rev = pd.concat([smart_read_csv(f) for f in rev_files], ignore_index=True)

 成功读取: /content/DAU Identity V(Mar 31, 2018 - Jul 13, 2025, 中国).csv, encoding=UTF-16, sep='	', shape=(6400, 13)
 成功读取: /content/DAU Justice (May 20, 2023 - Jul 13, 2025, 中国).csv, encoding=GB2312, sep=',', shape=(3070, 13)
 成功读取: /content/DAU Genshin Impact (Sep 25, 2020 - Jul 12, 2025, 中国).csv, encoding=UTF-16, sep='	', shape=(6064, 13)
 成功读取: /content/DAU Zenless Zero Zone (Jul 2, 2024 - Jul 12, 2025, 中国).csv, encoding=UTF-16, sep='	', shape=(900, 13)
 成功读取: /content/DAU Life After (Oct 31, 2018 - Jul 13, 2025, 中国).csv, encoding=UTF-16, sep='	', shape=(7074, 13)
 成功读取: /content/DAU Metal Slug Awakening(Apr 18, 2023 - Jul 13, 2025, 中国).csv, encoding=UTF-16, sep='	', shape=(388, 13)
 成功读取: /content/DAU Honor of King  (Oct 25, 2015 - Jul 13, 2025, 中国).csv, encoding=UTF-16, sep='	', shape=(8160, 13)
 成功读取: /content/DAU Dawn of Kingdoms (Mar 29, 2022 - Jul 13, 2025, 中国).csv, encoding=UTF-16, sep='	', shape=(2406, 13)
 成功读取: /content/DAU  PUBG Mobile (Feb 8, 2018 - Jul 13, 2025, 中国).csv, en

In [ ]:
df_dau.head()

,Unified Name,Unified ID,Unified Publisher Name,Unified Publisher ID,Publisher Name,Publisher ID,App Name,App ID,Date,Country / Region,Platform,Device,DAU
0,Identity V,5b2ba8cd8211f574a56c18fa,NetEase 网易,560c44888ac35064390048b9,网易移动游戏,656116440,第五人格,1330863325,2018-03-31,CN,App Store,iPhone,22037
1,Identity V,5b2ba8cd8211f574a56c18fa,NetEase 网易,560c44888ac35064390048b9,网易移动游戏,656116440,第五人格,1330863325,2018-03-31,CN,App Store,iPad,2958
2,Identity V,5b2ba8cd8211f574a56c18fa,NetEase 网易,560c44888ac35064390048b9,网易移动游戏,656116440,第五人格,1330863325,2018-04-01,CN,App Store,iPhone,202577
3,Identity V,5b2ba8cd8211f574a56c18fa,NetEase 网易,560c44888ac35064390048b9,网易移动游戏,656116440,第五人格,1330863325,2018-04-01,CN,App Store,iPad,66537
4,Identity V,5b2ba8cd8211f574a56c18fa,NetEase 网易,560c44888ac35064390048b9,网易移动游戏,656116440,第五人格,1330863325,2018-04-02,CN,App Store,iPhone,380838


In [ ]:
df_rev.head()

,Unified Name,Unified ID,Unified Publisher Name,Unified Publisher ID,Publisher Name,Publisher ID,App Name,App ID,Date,Country / Region,Platform,Device,Downloads,Revenue ($),RPD ($),ARPDAU ($)
0,Genshin Impact,5f3b76921392d052ebf0d0ca,miHoYo 米哈游,57ef5b4f0211a6aed3000136,miHoYo Games,1467190250,原神,1467190251,2020-09-25,CN,App Store,iPhone,312615,111.19,0.000356,0.000473
1,Genshin Impact,5f3b76921392d052ebf0d0ca,miHoYo 米哈游,57ef5b4f0211a6aed3000136,miHoYo Games,1467190250,原神,1467190251,2020-09-25,CN,App Store,iPad,38275,0.05,0.000001,0.000002
2,Genshin Impact,5f3b76921392d052ebf0d0ca,miHoYo 米哈游,57ef5b4f0211a6aed3000136,miHoYo Games,1467190250,原神,1467190251,2020-09-26,CN,App Store,iPhone,370936,80.33,0.000217,0.000186
3,Genshin Impact,5f3b76921392d052ebf0d0ca,miHoYo 米哈游,57ef5b4f0211a6aed3000136,miHoYo Games,1467190250,原神,1467190251,2020-09-26,CN,App Store,iPad,55455,1.76,0.000032,0.000029
4,Genshin Impact,5f3b76921392d052ebf0d0ca,miHoYo 米哈游,57ef5b4f0211a6aed3000136,miHoYo Games,1467190250,原神,1467190251,2020-09-27,CN,App Store,iPhone,364666,552480.97,1.515033,0.977552


###1.2 Data Cleanning

In [ ]:
# Check data info
def quick_check(df, name="Data"):
    print(f"=== {name} 数据质量检查 ===")
    print("\n--- 基本信息 ---")
    print(df.info())

    print("\n--- 缺失值统计 ---")
    print(df.isnull().sum())

    print("\n--- 重复行数 ---")
    print(df.duplicated().sum())

    print("\n--- 唯一值统计 ---")
    print(df.nunique())

    print("="*50)

#apply and check
quick_check(df_dau, "DAU数据")
quick_check(df_rev, "Revenue数据")


=== DAU数据 数据质量检查 ===

--- 基本信息 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42376 entries, 0 to 42375
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unified Name            42376 non-null  object
 1   Unified ID              42376 non-null  object
 2   Unified Publisher Name  42376 non-null  object
 3   Unified Publisher ID    42376 non-null  object
 4   Publisher Name          42376 non-null  object
 5   Publisher ID            42376 non-null  int64 
 6   App Name                42376 non-null  object
 7   App ID                  42376 non-null  int64 
 8   Date                    42376 non-null  object
 9   Country / Region        42376 non-null  object
 10  Platform                42376 non-null  object
 11  Device                  42376 non-null  object
 12  DAU                     42376 non-null  int64 
dtypes: int64(3), object(10)
memory usage: 4.2+ MB
None

--- 缺失值统计 ---
Unifi

In [ ]:
#to datetime
df_dau['Date'] = pd.to_datetime(df_dau['Date'], errors='coerce')
df_rev['Date'] = pd.to_datetime(df_rev['Date'], errors='coerce')

In [ ]:
#Clean Data

def clean_dataframe(df):
    #check before
    print(f"\n原始行数: {len(df)}")
    print(f"缺失值统计（清理前）:\n{df.isna().sum()}")
    print(f"重复行数（清理前）: {df.duplicated().sum()}")

    # 删除重复行
    df = df.drop_duplicates()

    # 缺失值填充
    for col in df.columns:
        if df[col].isna().any():
            if df[col].dtype in ['int64', 'float64']:
                df[col] = df[col].fillna(0)
            else:
                df[col] = df[col].fillna("Unknown")

    #check after
    print(f"\n清理后行数: {len(df)}")
    print(f"缺失值统计（清理后）:\n{df.isna().sum()}")
    print(f"重复行数（清理后）: {df.duplicated().sum()}")
    print("="*50)

    return df


#apply
df_dau = clean_dataframe(df_dau)
df_rev = clean_dataframe(df_rev)


原始行数: 42376
缺失值统计（清理前）:
Unified Name                 0
Unified ID                   0
Unified Publisher Name       0
Unified Publisher ID         0
Publisher Name               0
Publisher ID                 0
App Name                     0
App ID                       0
Date                      5558
Country / Region             0
Platform                     0
Device                       0
DAU                          0
dtype: int64
重复行数（清理前）: 127

清理后行数: 42249
缺失值统计（清理后）:
Unified Name              0
Unified ID                0
Unified Publisher Name    0
Unified Publisher ID      0
Publisher Name            0
Publisher ID              0
App Name                  0
App ID                    0
Date                      0
Country / Region          0
Platform                  0
Device                    0
DAU                       0
dtype: int64


/tmp/ipython-input-2825552302.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna("Unknown")


重复行数（清理后）: 0

原始行数: 45282
缺失值统计（清理前）:
Unified Name                  0
Unified ID                    0
Unified Publisher Name        0
Unified Publisher ID          0
Publisher Name                0
Publisher ID                  0
App Name                      0
App ID                        0
Date                      10152
Country / Region              0
Platform                      0
Device                        0
Downloads                     0
Revenue ($)                   0
RPD ($)                     381
ARPDAU ($)                 2877
dtype: int64
重复行数（清理前）: 0

清理后行数: 45282
缺失值统计（清理后）:
Unified Name              0
Unified ID                0
Unified Publisher Name    0
Unified Publisher ID      0
Publisher Name            0
Publisher ID              0
App Name                  0
App ID                    0
Date                      0
Country / Region          0
Platform                  0
Device                    0
Downloads                 0
Revenue ($)               0
RPD ($

In [ ]:
#check data
display(df_dau.columns)
display(df_rev.columns)

Index(['Unified Name', 'Unified ID', 'Unified Publisher Name',
       'Unified Publisher ID', 'Publisher Name', 'Publisher ID', 'App Name',
       'App ID', 'Date', 'Country / Region', 'Platform', 'Device', 'DAU'],
      dtype='object')

Index(['Unified Name', 'Unified ID', 'Unified Publisher Name',
       'Unified Publisher ID', 'Publisher Name', 'Publisher ID', 'App Name',
       'App ID', 'Date', 'Country / Region', 'Platform', 'Device', 'Downloads',
       'Revenue ($)', 'RPD ($)', 'ARPDAU ($)'],
      dtype='object')

In [ ]:
#select core columns
df_dau = df_dau[['Unified Name','Unified ID','Publisher Name','Date','DAU']]
df_rev = df_rev[['Unified Name','Unified ID','Publisher Name','Date','Revenue ($)']]

In [ ]:
# aggregate DAU
agg_dau = df_dau.groupby(['Unified ID', 'Date'], as_index=False).agg({
    'Unified Name': 'first',       # 取第一条记录的游戏名作为聚合后记录的游戏名
    'Publisher Name': 'first',
    'DAU': 'sum'
})

# aggregate Revenue
agg_rev = df_rev.groupby(['Unified ID', 'Date'], as_index=False).agg({
    'Unified Name': 'first',
    'Publisher Name': 'first',
    'Revenue ($)': 'sum'
})


In [ ]:
#merge DAU and Revenue
df_merged = pd.merge(agg_dau, agg_rev, on=['Unified Name', 'Date'], how='inner')

In [ ]:
#final cleanning
df_merged = df_merged[(df_merged['DAU'] >= 0) & (df_merged['Revenue ($)'] >= 0)]

In [ ]:
df_cleaned = df_merged
df_cleaned

,Unified ID_x,Date,Unified Name,Publisher Name_x,DAU,Unified ID_y,Publisher Name_y,Revenue ($)
0,5b2ba8cd8211f574a56c18fa,2018-03-31 00:00:00,Identity V,网易移动游戏,24995,5b2ba8cd8211f574a56c18fa,网易移动游戏,0.00
1,5b2ba8cd8211f574a56c18fa,2018-04-01 00:00:00,Identity V,网易移动游戏,269114,5b2ba8cd8211f574a56c18fa,网易移动游戏,1.59
2,5b2ba8cd8211f574a56c18fa,2018-04-02 00:00:00,Identity V,网易移动游戏,500021,5b2ba8cd8211f574a56c18fa,网易移动游戏,81659.92
3,5b2ba8cd8211f574a56c18fa,2018-04-03 00:00:00,Identity V,网易移动游戏,669288,5b2ba8cd8211f574a56c18fa,网易移动游戏,137650.62
4,5b2ba8cd8211f574a56c18fa,2018-04-04 00:00:00,Identity V,网易移动游戏,816769,5b2ba8cd8211f574a56c18fa,网易移动游戏,147156.63
...,...,...,...,...,...,...,...,...
12380,6626a822a9cba85698402d6b,2025-05-03 00:00:00,Zenless Zone Zero - 1st Anniv.,miHoYo Games,63625,6626a822a9cba85698402d6b,miHoYo Games,35881.99
12381,6626a822a9cba85698402d6b,2025-05-04 00:00:00,Zenless Zone Zero - 1st Anniv.,miHoYo Games,46433,6626a822a9cba85698402d6b,miHoYo Games,30603.62
12382,6626a822a9cba85698402d6b,2025-05-05 00:00:00,Zenless Zone Zero - 1st Anniv.,miHoYo Games,32801,6626a822a9cba85698402d6b,miHoYo Games,25272.45
12383,6626a822a9cba85698402d6b,2025-05-06 00:00:00,Zenless Zone Zero - 1st Anniv.,miHoYo Games,26231,6626a822a9cba85698402d6b,miHoYo Games,30379.13


In [ ]:
df_cleaned.to_csv("Cleaned_DAU_Revenue.csv", index=False)

###1.3 Rename Game Names

In [ ]:
#import
data = pd.read_csv("/content/Cleaned_DAU_Revenue.csv")
display(data.columns)

Index(['Unified ID_x', 'Date', 'Unified Name', 'Publisher Name_x', 'DAU',
       'Unified ID_y', 'Publisher Name_y', 'Revenue ($)'],
      dtype='object')

In [ ]:
# Display all unique values in the 'Unified Name' column
display(data['Unified Name'].unique())

array(['Identity V', 'Genshin Impact', '重返帝国-帝国时代正版合作',
       'Metal Slug: Awakening', 'Justice Mobile', 'Honor of Kings',
       '和平精英', 'Zenless Zone Zero - 1st Anniv.'], dtype=object)

In [ ]:
#rename collumn
data = data.rename(columns={'Unified Name': 'Name', 'Date': 'Date', 'Revenue ($)': 'Revenue'})

#select data for modolling
md_data = data[['Name', 'Date', 'DAU', 'Revenue']]
md_data

,Name,Date,DAU,Revenue
0,Identity V,2018-03-31 00:00:00,24995,0.00
1,Identity V,2018-04-01 00:00:00,269114,1.59
2,Identity V,2018-04-02 00:00:00,500021,81659.92
3,Identity V,2018-04-03 00:00:00,669288,137650.62
4,Identity V,2018-04-04 00:00:00,816769,147156.63
...,...,...,...,...
12380,Zenless Zone Zero - 1st Anniv.,2025-05-03 00:00:00,63625,35881.99
12381,Zenless Zone Zero - 1st Anniv.,2025-05-04 00:00:00,46433,30603.62
12382,Zenless Zone Zero - 1st Anniv.,2025-05-05 00:00:00,32801,25272.45
12383,Zenless Zone Zero - 1st Anniv.,2025-05-06 00:00:00,26231,30379.13


In [ ]:
#replace chinese name
name_map = {
    "明日之后": "LifeAfter",
    "王者荣耀": "Honor of Kings",
    "和平精英": "PUBG Mobile",
    "逆水寒": "Justice Mobile",
    "重返帝国-帝国时代正版合作": "Dawn of kingdoms"
}

md_data['Name'] = md_data['Name'].replace(name_map)

#check data
display(md_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 11929 entries, 1 to 12384
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Name     11929 non-null  object 
 1   Date     11929 non-null  object 
 2   DAU      11929 non-null  int64  
 3   Revenue  11929 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 466.0+ KB


/tmp/ipython-input-2904732518.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  md_data['Name'] = md_data['Name'].replace(name_map)


None

In [ ]:
md_data.to_csv("model data.csv", index=False)